In [ ]:
!nvcc --version # cuda already installed
!nvidia-smi # nvidia drivers already installed

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2020 NVIDIA Corporation
Built on Mon_Oct_12_20:09:46_PDT_2020
Cuda compilation tools, release 11.1, V11.1.105
Build cuda_11.1.TC455_06.29190527_0
Sun Aug 14 14:52:11 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |            

In [ ]:
!pip install git+https://github.com/andreinechaev/nvcc4jupyter.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-euvp1s_y
  Running command git clone -q https://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-euvp1s_y
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-py3-none-any.whl size=4306 sha256=bc970e0c9eae3c683772abe21708ffe85692d767bdf24c497fc31dce39f7b1f4
  Stored in directory: /tmp/pip-ephem-wheel-cache-zyk748ic/wheels/ca/33/8d/3c86eb85e97d2b6169d95c6e8f2c297fdec60db6e84cb56f5e
Successfully built NVCCPlugin


In [ ]:
%load_ext nvcc_plugin

created output directory at /content/src
Out bin /content/result.out


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')


Mounted at /content/gdrive


In [ ]:
!pip install wandb
import wandb
wandb.init()

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.8 MB 8.2 MB/s 
     |████████████████████████████████| 157 kB 74.4 MB/s 
     |████████████████████████████████| 181 kB 73.3 MB/s 
     |████████████████████████████████| 63 kB 2.4 MB/s 
     |████████████████████████████████| 157 kB 66.1 MB/s 
     |████████████████████████████████| 157 kB 80.9 MB/s 
     |████████████████████████████████| 157 kB 60.5 MB/s 
     |████████████████████████████████| 156 kB 76.5 MB/s 
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8806 sha256=7cd348892cb1277c8d9eef9f0703fc13052b35ea5bf9be19f97af952db467199
  Stored in directory: /root/.cache/pip/wheels/3e/31/09/fa59cef12cdcfecc627b3d24273699f390e71828921b2cbba2
Successfully built pathtools


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 

··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [8]:

%%cu
#include <iostream>
int main()
{
    std::cout << "Welcome To GeeksforGeeks\n";
    return 0;
}

Welcome To GeeksforGeeks



My code

In [10]:
%%cu
#include "cuda_runtime.h"
#include "device_launch_parameters.h"
//#include "data_header.h"
#include "cuda.h"
#ifndef __CUDACC__  
#define __CUDACC__
#endif
#include <device_functions.h>
#include <curand.h>
#include <curand_kernel.h>

#include <stdio.h>
#include <string>
#include <vector>
#include <iostream>
#include <fstream>
#include <random>
#include <chrono>


using namespace std;

vector<string> tokenize(string const& str, const char* delim);
__global__ void mcmc_block(float* vars, int* n_vars, float* state, float* min_state, float* min_energy, float* energy, float* proposal_state, int* rng, float* distogram, int* CA, int* CB, int* C, int* N, int* O, float* e_history, int* min_reset_ind, float *min_reset_energy, bool* global_halt, float* energy_target);

static const int n_threads_const = 512;
static const int n_blocks_const = 50;



int main()
{
    // User parameters //////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////
    const int n_runs = 100;                                         // number of runs to do, each with a different seed. Each run saves to a different file
    const int n_blocks = n_blocks_const;                           // number of parallel mcmc blocks
    const int n_threads = n_threads_const;                         // number of threads per block
    const int n_cycles = 10;                                       // cycles to repeat all ranges (for alt-mh)
    const float beta = 0.1;
    const int n_rngs = 2;                                          // number of rng seeds required per proposal (protein folding requires one for gaussian noise & one for criterion)
    vector<float> noise_var_range = { 10, 9 };                     // noise added to current state shrinks from n_v_r[0] to n_v_r[1], so later proposals are tigher 
    vector<int> mh_ranges = { 16, 10000 };                            // ranges to alternate between
    vector<int> mh_iter_per_range = { 50, 50 };                    // iterations per corresponding range. each iteration is n_residues*2 proposals, centered on each bond-of-interest 
    
    const float energy_target_usr = 0;                           // stop the gpu once it finds this energy among ANY block; GPU will run for this or cycles, whichever comes first

    string data_filename("/content/gdrive/MyDrive/Colab Notebooks/Molecular_Simulations/GPU_for_pf/data/gpudata_T0955.csv"); // google colab
    string out_folder("/content/gdrive/MyDrive/Colab Notebooks/Molecular_Simulations/GPU_for_pf/results"); // google colab
    ///////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////

    // read in data
    vector<float> build_distogram, build_log_distogram, build_state, build_energy;
    vector<vector<int>> build_atom_ind{ {},{},{},{},{},{} };
    int n = 0, linenum = 0, n_ranges = mh_ranges.size();

    vector<string> split_line;
    ifstream ddrfile(data_filename);
    if (ddrfile.is_open())
    {
        string line;
        while (getline(ddrfile, line))
        {
            cout << "linenum: " << linenum << " ";
            split_line = tokenize(line, ",");
            cout << "number of tokens : " << split_line.size() << endl;
            n = 0;
            while (n < split_line.size())
            {
                if (linenum == 0)       // dist
                    build_distogram.push_back(stof(split_line[n]));
                else if (linenum == 1)  // initial_state
                    build_state.push_back(stof(split_line[n]));
                else if (linenum >= 2)  // indices for CA, CB, C, N, O, H in that order
                    build_atom_ind[linenum - 2].push_back(stof(split_line[n]));
                n++;
            }
            
            linenum++;
        }
        cout << "[log] successfully read in the data!" << endl;
    }
    else
        cout << "[error] error opening ddrfile" << endl;


    int n_residues = build_atom_ind[0].size();
    int n_atoms = build_state.size() / 3;
    int size_distogram = build_distogram.size();
    int size_state = build_state.size();
    build_energy.push_back(100000000);
    vector<float> state_copy(build_state);

    for (int i = 1; i < n_blocks; i++)
    {
        build_state.insert(build_state.end(), state_copy.begin(), state_copy.end());
        build_energy.push_back(100000000);
    }
    float* energy = &build_energy[0];
    int max_mh_range = 3;//*max_element(mh_ranges.begin(), mh_ranges.end());
    float* distogram = &build_distogram[0];
    float* state = &build_state[0];
    int* ca_ind = &build_atom_ind[0][0];
    int* cb_ind = &build_atom_ind[1][0];
    int* c_ind = &build_atom_ind[2][0];
    int* n_ind = &build_atom_ind[3][0];
    int* o_ind = &build_atom_ind[2][0];
    int* h_ind = &build_atom_ind[3][0];
    float*energy_target = new float(energy_target_usr);
 
    // error checking
    if (n_threads % 32 != 0)
        cerr << "[WARNING] use a multiple of 32 for n_threads for optimal performance." << endl;
    if (mh_ranges.size() != mh_iter_per_range.size())
    {
        cerr << "[ERROR] mh_ranges and mh_iter_per_range must be the same size." << endl;
        return 0;
    }
    if (noise_var_range.size() != 2)
    {
        cerr << "[ERROR] noise_var_range must have two values (a,b), a>b where proposals will be generated via gaussian noise with var=a, and reduce to var=b over time." << endl;
        return 0;
    }
    if (n_blocks <= 0 || n_threads <= 0)
    {
        cerr << "[ERROR] n_blocks and n_threads must be >= 1. n_blocks determines how many mcmc chains to run, and n_threads determines how much to parallelize each chain" << endl;
        return 0;
    }
    float avg_time=0;
    int angle = 0;
    int n_iters = 0;
    for (int i = 0; i < n_ranges; i++)
        n_iters += mh_iter_per_range[i];
    int n_proposals = n_cycles * n_iters * n_residues * 2;
    int min_ind = 0;
    int min_val = 1000000;
    // combine loose variables into one array to efficiently send to GPU (add vars as needed)
    vector <float> build_vars;
    build_vars.push_back(beta); // 0
    build_vars.push_back(n_residues); // 1
    build_vars.push_back(n_atoms); // 2
    build_vars.push_back(energy[0]); // 3
    build_vars.push_back(n_cycles);// 4
    build_vars.push_back(n_ranges); // 5
    build_vars.push_back(n_rngs); // 6
    build_vars.push_back(angle); // 7
    build_vars.push_back(max_mh_range); // 8
    build_vars.insert(build_vars.end(), noise_var_range.begin(), noise_var_range.end()); // 9
    build_vars.insert(build_vars.end(), mh_ranges.begin(), mh_ranges.end()); // 11
    build_vars.insert(build_vars.end(), mh_iter_per_range.begin(), mh_iter_per_range.end());// 11+n_ranges

    int n_vars = build_vars.size();
    float* vars = &build_vars[0];

    // initialize pointers to device memory  
    cudaError_t cudaStatus;
    float* dev_distogram, * dev_state, * dev_min_state, * dev_min_energy, * dev_proposal_state, * dev_vars, * dev_state_energies, * dev_prop_state_energies, * dev_energy, * dev_e_history, * dev_min_reset_energy;
    int* dev_ca, * dev_cb, * dev_c, * dev_n, * dev_o, * dev_h, * dev_rng, * dev_n_vars, * p_n_vars = &n_vars, *dev_min_reset_ind;
    float* dev_gaus_rng, *dev_energy_target;
    int n_energies = ((n_residues - 1) * n_residues) / 2;
    bool *global_halt = new bool(false);
    bool *dev_global_halt;


    // reserve memory on the device
    cudaStatus = cudaMalloc((void**)&dev_rng, n_rngs * n_blocks * sizeof(int));
    if (cudaStatus != cudaSuccess) {
        fprintf(stderr, "cudaMalloc failed!");
        goto Error;
    }
    cudaStatus = cudaMalloc((void**)&dev_distogram, size_distogram * sizeof(float));
    cudaStatus = cudaMalloc((void**)&dev_state, size_state * n_blocks * sizeof(float));
    cudaStatus = cudaMalloc((void**)&dev_min_state, size_state * n_blocks * sizeof(float));
    cudaStatus = cudaMalloc((void**)&dev_min_energy, n_blocks * sizeof(float));
    cudaStatus = cudaMalloc((void**)&dev_proposal_state, size_state * n_blocks * sizeof(float));
    cudaStatus = cudaMalloc((void**)&dev_ca, n_residues * sizeof(int));
    cudaStatus = cudaMalloc((void**)&dev_cb, n_residues * sizeof(int));
    cudaStatus = cudaMalloc((void**)&dev_c, n_residues * sizeof(int));
    cudaStatus = cudaMalloc((void**)&dev_n, n_residues * sizeof(int));
    cudaStatus = cudaMalloc((void**)&dev_o, n_residues * sizeof(int));
    cudaStatus = cudaMalloc((void**)&dev_h, n_residues * sizeof(int));
    cudaStatus = cudaMalloc((void**)&dev_vars, n_vars * sizeof(int));
    cudaStatus = cudaMalloc((void**)&dev_n_vars, sizeof(int));
    cudaStatus = cudaMalloc((void**)&dev_state_energies, n_energies * sizeof(float));
    cudaStatus = cudaMalloc((void**)&dev_prop_state_energies, n_energies * sizeof(float));
    cudaStatus = cudaMalloc((void**)&dev_energy, n_blocks * sizeof(float));
    cudaStatus = cudaMalloc((void**)&dev_e_history, n_blocks * n_proposals * sizeof(float));
    cudaStatus = cudaMalloc((void**)&dev_min_reset_ind, sizeof(int));
    cudaStatus = cudaMalloc((void**)&dev_min_reset_energy, sizeof(float));
    cudaStatus = cudaMalloc((void**)&dev_global_halt, sizeof(bool));
    cudaStatus = cudaMalloc((void**)&dev_energy_target, sizeof(float));


    for (int run = 0; run < n_runs; run++)
    {
        // initialize RNG
        int rand_max = (1u << 31) - 1; // 2^32-1, should fill a 32 bit register
        std::default_random_engine generator;
        std::uniform_int_distribution<int> distribution(0, rand_max);
        generator.seed(std::chrono::system_clock::now().time_since_epoch().count()); 
        vector<int> build_rng;
        for (int i = 0; i < n_rngs * n_blocks; i++)
            build_rng.push_back(distribution(generator));
        int* rng = &build_rng[0];

        // copy data into the reserved device memory
        cudaStatus = cudaMemcpy(dev_rng, rng, n_rngs * n_blocks * sizeof(int), cudaMemcpyHostToDevice);
        if (cudaStatus != cudaSuccess) {
            fprintf(stderr, "cudaMemcpy failed!");
            goto Error;
        }
        cudaStatus = cudaMemcpy(dev_distogram, distogram, size_distogram * sizeof(float), cudaMemcpyHostToDevice);
        cudaStatus = cudaMemcpy(dev_state, state, size_state * n_blocks * sizeof(float), cudaMemcpyHostToDevice);
        cudaStatus = cudaMemcpy(dev_min_state, state, size_state * n_blocks * sizeof(float), cudaMemcpyHostToDevice);
        cudaStatus = cudaMemcpy(dev_proposal_state, state, size_state * n_blocks * sizeof(float), cudaMemcpyHostToDevice);
        cudaStatus = cudaMemcpy(dev_ca, ca_ind, n_residues * sizeof(int), cudaMemcpyHostToDevice);
        cudaStatus = cudaMemcpy(dev_cb, cb_ind, n_residues * sizeof(int), cudaMemcpyHostToDevice);
        cudaStatus = cudaMemcpy(dev_c, c_ind, n_residues * sizeof(int), cudaMemcpyHostToDevice);
        cudaStatus = cudaMemcpy(dev_n, n_ind, n_residues * sizeof(int), cudaMemcpyHostToDevice);
        cudaStatus = cudaMemcpy(dev_o, o_ind, n_residues * sizeof(int), cudaMemcpyHostToDevice);
        cudaStatus = cudaMemcpy(dev_h, h_ind, n_residues * sizeof(int), cudaMemcpyHostToDevice);
        cudaStatus = cudaMemcpy(dev_vars, vars, n_vars * sizeof(float), cudaMemcpyHostToDevice);
        cudaStatus = cudaMemcpy(dev_n_vars, p_n_vars, sizeof(int), cudaMemcpyHostToDevice);
        cudaStatus = cudaMemcpy(dev_energy, energy, n_blocks * sizeof(float), cudaMemcpyHostToDevice);
        cudaStatus = cudaMemcpy(dev_min_energy, energy, n_blocks * sizeof(float), cudaMemcpyHostToDevice);
        cudaStatus = cudaMemcpy(dev_global_halt, global_halt, sizeof(bool), cudaMemcpyHostToDevice);
        cudaStatus = cudaMemcpy(dev_energy_target, energy_target, sizeof(float), cudaMemcpyHostToDevice);


        // shared memory struct:
        //      4 arrays of size n_threads_const for summing energies
        //      state array - n_atoms*3
        //      proposal array - n_atoms*3

        unsigned int dyn_shared_memsize = (4*n_threads_const + 2 * size_state) * sizeof(float);
        cout << "[log] bytes of dynamic shared memory: " << dyn_shared_memsize << endl;
        auto start = chrono::steady_clock::now();
        mcmc_block << < n_blocks, n_threads, dyn_shared_memsize >> > (
            dev_vars,
            dev_n_vars,

            dev_state,
            dev_min_state,
            dev_min_energy,
            dev_energy,
            dev_proposal_state,

            dev_rng,

            dev_distogram,
            dev_ca,
            dev_cb,
            dev_c,
            dev_n,
            dev_o,

            dev_e_history,
            dev_min_reset_ind,
            dev_min_reset_energy,
            dev_global_halt,
            dev_energy_target);

        // Check for any errors launching the kernel
        cudaStatus = cudaGetLastError();
        if (cudaStatus != cudaSuccess) {
            fprintf(stderr, "mcmc_block kernel launch failed: %s\n", cudaGetErrorString(cudaStatus));
            goto Error;
        }


        // cudaDeviceSynchronize waits for the kernel to finish
        cudaStatus = cudaDeviceSynchronize();
        if (cudaStatus != cudaSuccess) {
            fprintf(stderr, "cudaDeviceSynchronize returned error code %d after launching kernel!\n", cudaStatus);
            goto Error;
        }

        auto end = chrono::steady_clock::now();
        int time_ms = chrono::duration_cast<chrono::milliseconds>(end - start).count();
        cout << "Elapsed time in milliseconds: " << time_ms << " ms" << endl;
        avg_time += time_ms;

        // Copy results from GPU buffer to host memory.
        float* read_out_state = new float[size_state * n_blocks];
        cudaStatus = cudaMemcpy(read_out_state, dev_state, size_state * n_blocks * sizeof(float), cudaMemcpyDeviceToHost);
        if (cudaStatus != cudaSuccess) {
            fprintf(stderr, "cudaMemcpy state failed!");
            goto Error;
        }
        float* read_out_energy = new float[n_blocks];
        cudaStatus = cudaMemcpy(read_out_energy, dev_energy, n_blocks * sizeof(float), cudaMemcpyDeviceToHost);
        float* e_history = new float[n_blocks * n_proposals];
        cudaStatus = cudaMemcpy(e_history, dev_e_history, n_blocks * n_proposals * sizeof(float), cudaMemcpyDeviceToHost);


        // find minimum energy
        float min_energy = read_out_energy[0];
        int min_block = 0;
        for (int i = 0; i < n_blocks; i++)
            if (read_out_energy[i] < min_energy)
            {
                min_energy = read_out_energy[i];
                min_block = i;
            }

        // write that energy and state to file
        string fnum = std::to_string(run);
        string fname = out_folder;
        fname.append("/results");
        fname.append(fnum);
        fname.append(".txt");

        ofstream myfile(fname);
        if (!myfile.is_open())
            cout << "error opening file on run: " << run << endl;
        myfile << n_blocks << endl;
        myfile << read_out_state[0 + min_block * n_atoms * 3];
        for (int ii = 1; ii < size_state; ii++)
            myfile << ", " << read_out_state[ii + min_block * n_atoms * 3];
        myfile << endl;
        for (int ii = 0; ii < n_blocks; ii++)
        {
            myfile << e_history[ii * n_proposals];
            for (int i = 0; i < n_proposals; i++)
                myfile << ", " << e_history[i + ii * n_proposals];
            myfile << endl;
        }
        myfile << read_out_energy[min_block] << endl;
        myfile << time_ms << endl;
        myfile.close();


        cout << "[log] host min_energy: " << read_out_energy[min_block] << endl;
    }
    cout << "[log] average time over " << n_runs << " runs is : "<< avg_time / n_runs << "ms" << endl;

Error:
    cudaFree(dev_distogram);
    cudaFree(dev_state);
    cudaFree(dev_ca);
    cudaFree(dev_cb);
    cudaFree(dev_c);
    cudaFree(dev_n);
    cudaFree(dev_o);
    cudaFree(dev_h);
    cudaFree(dev_vars);
    cudaFree(dev_rng);
    cudaFree(dev_state_energies);
    cudaFree(dev_prop_state_energies);
    cudaFree(dev_energy);



    // cudaDeviceReset must be called before exiting in order for profiling and
    // tracing tools such as Nsight and Visual Profiler to show complete traces.
    cudaStatus = cudaDeviceReset();
    if (cudaStatus != cudaSuccess) {
        fprintf(stderr, "cudaDeviceReset failed!");
        return 1;
    }

    return 0;
}

// Helper function to tokenize a line
vector<string> tokenize(string const& str, const char* delim)
{
    char* token = strtok(const_cast<char*>(str.c_str()), delim);
    vector<string> split_line;

    while (token != nullptr)
    {
        split_line.push_back(string(token));
        token = strtok(nullptr, delim);
    }
    return split_line;
}


// kernels
__global__ void mcmc_block(float* vars, int* n_vars, float* state, float* min_state, float* min_energy, float* energy, float* proposal_state, int* rng, float* distogram, int* CA, int* CB, int* C, int* N, int* O, float* e_history, int* min_reset_ind, float* min_reset_energy, bool* global_halt, float*energy_target)
{
    int t = threadIdx.x;
    int stride = blockDim.x;
    int b = blockIdx.x;
    int n_blocks = gridDim.x;

    // allocate local & shared memory --> https://developer.nvidia.com/blog/using-sh
    int n_residues = vars[1], n_atoms = vars[2], n_cycles = vars[4], n_ranges = vars[5], n_rngs = vars[6], angle = vars[7], max_mh_range = vars[8], n_angles = n_residues * 2;
    float beta = vars[0];
    int n_energies = ((n_residues - 1) * n_residues) / 2, ii;
    float* noise_range, * mh_ranges, * mh_iter_per_range;
    float point[3], rotated_point[3], rngfloat, two_16 = 65536, two_15 = 32768, two_14 = 16384;
    float two_31 = 2147483648, two_30 = 1073741824, two_32=4294967296;

    clock_t start_time = clock(); 
    curandState_t custate;
    curandStatePhilox4_32_10_t custate_philox;
    curandStateMRG32k3a_t custate_mrg32k3a;
    curandStateXORWOW_t custate_xorwow;
    



    __shared__ float rng_shared[2], ax_vec[3], unit_ax_vec[3], offset[3], norm, cos_rad, sin_rad, noise_var, noise_var_del;
    extern __shared__ float dyn_shared_mem[];
    float* sums_state = (float*)dyn_shared_mem;
    float* sums_glob_state = (float*)&sums_state[n_threads_const];
    float* sums_prop_state = (float*)&sums_glob_state[n_threads_const];
    float* sums_glob_prop_state = (float*)&sums_prop_state[n_threads_const];
    float* state_shared = (float*)&sums_glob_prop_state[n_threads_const];
    float* proposal_state_shared = (float*)&state_shared[n_atoms*3];
    
    __shared__ int proposal, n_proposals, n_iters, n_accepted;//, rng_stored[2];
    __shared__ bool accept, lowest;
    __shared__ float start_tid, end_tid;
    __shared__ int rng_stored[2];
 
    proposal = 0;
    n_accepted = 0;
    noise_range = &vars[9];
    mh_ranges = &vars[11];
    mh_iter_per_range = &vars[11 + n_ranges];
    n_iters = 0;

    if (t == 0)
        for (ii = 0; ii < n_ranges; ii++)
            n_iters += mh_iter_per_range[ii];
    for (ii = 0; ii < n_atoms * 3; ii++)
    {    
        state_shared[ii] = state[ii + b*n_atoms*3];
        proposal_state_shared[ii] = state[ii + b*n_atoms*3];
        min_state[ii + b * n_atoms * 3] = state[ii + b * n_atoms * 3];
        
    }
    for (ii = 0; ii < n_threads_const; ii++)
    {
        sums_state[ii] = 0;
        sums_prop_state[ii] = 0;
        sums_glob_state[ii] = 0;
        sums_glob_prop_state[ii] = 0;
    }
    if (t == 0)
        for (ii = 0; ii < n_rngs; ii++)
            rng_stored[ii] = rng[ii + b * n_rngs];

    if (t < n_rngs)
    {
        curand_init(rng_stored[t],  /* the seed controls the sequence of random values that are produced */
              0,                    /* the sequence number is only important with multiple cores */
              0,                    /* the offset is how much extra we advance in the sequence for each call, can be 0 */
              &custate_xorwow);
    }
    if (t==0)
    {    
        n_proposals = n_cycles * n_iters * n_residues * 2;
        noise_var_del = (noise_range[0] - noise_range[1]) / n_proposals * 3.14159265/180.0;
        noise_var = noise_range[0] * 3.14159265/180.0 - noise_var_del;
    }
    min_energy[b] = 1000000;

    /*if (t == 0 & b == 0)
    {
        printf("[log] n_energies: %d\n", n_energies);
        printf("[log] noise_var_range: %d --> %d\n", noise_range[0], noise_range[1]);
        printf("[log] noise_var: %f, del: %f\n", noise_var, noise_var_del);
        printf("[log] cycles: %d\n", n_cycles);
        for (ii = 0; ii < n_ranges; ii++)
            printf("[log] range: %f, iterations: %f\n", mh_ranges[ii], mh_iter_per_range[ii]);
    }*/
    __syncthreads();

    // begin mcmc
    for (int cycle = 0; cycle < n_cycles; cycle++)
    {
        //if (t == 0)
            //printf("[log] cycle: %d, noise_var: %f, n_accepted: %d/%d, min_energy: %f\n", cycle, noise_var, n_accepted, proposal, min_energy[b]);
        if (*global_halt)
            break;
        for (int range_sel = 0; range_sel < n_ranges; range_sel++)
        {
            if (*global_halt)
                break;
            int mh_range = mh_ranges[range_sel];
            for (int iter = 0; iter < mh_iter_per_range[range_sel]; iter++)
            {
                if (*global_halt)
                        break;
                for (int res = 0; res < n_residues; res++)
                {
                    if (*global_halt)
                        break;
                    for (int angle = 0; angle < 2; angle++)
                    {
                        if (*global_halt)
                            break;
                        // ------------------------------ pbit_logic: rng --------------------------------------------------------------------------------------------------------------
                        //different seed for every cuda core
                        if (t == 0)
                        {   
                            rng_shared[0] = curand_uniform(&custate_xorwow);
                        }
                        if (t == 1)
                        {
                            rng_shared[1] = curand_normal(&custate_xorwow) * noise_var;
                            //noise_var -= noise_var_del;
                            //printf("rng_shared[%d]: %f\n", t, rng_shared[t]);
                        }
                        // ------------------------------ end pbit_logic ---------------------------------------------------------------------------------------------------------------
                        __syncthreads();
                        // ------------------------------ proposal --------------------------------------------------------------------------------------------------------------------

                        if (t == 0)
                        {
                            if (angle == 0) //phi
                            {
                                for (ii = 0; ii < 3; ii++)
                                    ax_vec[ii] = state_shared[N[res] * 3 + ii] - state_shared[CA[res] * 3 + ii]; // added + b*n_atoms*3 to account for each block having its own state
                                norm = sqrtf(ax_vec[0] * ax_vec[0] + ax_vec[1] * ax_vec[1] + ax_vec[2] * ax_vec[2]);
                                for (ii = 0; ii < 3; ii++)
                                {
                                    unit_ax_vec[ii] = ax_vec[ii] / norm;
                                    offset[ii] = state_shared[N[res] * 3 + ii];
                                }
                            }
                            else if (angle == 1) //psi
                            {
                                for (ii = 0; ii < 3; ii++)
                                    ax_vec[ii] = state_shared[C[res] * 3 + ii] - state_shared[CA[res] * 3 + ii];
                                norm = sqrtf(ax_vec[0] * ax_vec[0] + ax_vec[1] * ax_vec[1] + ax_vec[2] * ax_vec[2]);
                                for (ii = 0; ii < 3; ii++)
                                {
                                    unit_ax_vec[ii] = ax_vec[ii] / norm;
                                    offset[ii] = state_shared[C[res] * 3 + ii];
                                }
                            }
                            cos_rad = cosf(rng_shared[1]);
                            sin_rad = sinf(rng_shared[1]);
                        }
                        __syncthreads();

                        for (int tid = t; tid < n_atoms; tid += stride)
                        {
                            // initialize proposal state as current state
                            for (ii = 0; ii < 3; ii++)
                            {
                                proposal_state_shared[tid * 3 + ii] = state_shared[tid * 3 + ii];
                                point[ii] = state_shared[tid * 3 + ii] - offset[ii];
                            }

                            float term1 = (unit_ax_vec[0] * point[0] + unit_ax_vec[1] * point[1] + unit_ax_vec[2] * point[2]) * (1 - cos_rad);

                            rotated_point[0] = (unit_ax_vec[0] * term1 + point[0] * cos_rad + (-1 * unit_ax_vec[2] * point[1] + unit_ax_vec[1] * point[2]) * sin_rad);
                            rotated_point[1] = (unit_ax_vec[1] * term1 + point[1] * cos_rad + (1 * unit_ax_vec[2] * point[0] - unit_ax_vec[0] * point[2]) * sin_rad);
                            rotated_point[2] = (unit_ax_vec[2] * term1 + point[2] * cos_rad + (-1 * unit_ax_vec[1] * point[0] + unit_ax_vec[0] * point[1]) * sin_rad);

                            // if phi, replace any points before N[j] with rotated points 
                            if (angle == 0 && tid < N[res])
                                for (int jj = 0; jj < 3; jj++)
                                    proposal_state_shared[tid * 3 + jj] = rotated_point[jj] + offset[jj];

                            // if psi, replace any points after N[j+1] with rotated points 
                            else if (angle == 1 && tid >= N[res + 1])
                                for (int jj = 0; jj < 3; jj++)
                                    proposal_state_shared[tid * 3 + jj] = rotated_point[jj] + offset[jj];
                        }

                        // ------------------------------ end proposal ------------------------------------------------------------------------------------------------------------------
                        __syncthreads();
                        // ------------------------------ energy ------------------------------------------------------------------------------------------------------------------------

                        // calculate full energy for min_energy state tracking, but use only a subsection for accept/reject
                        int start;
                        int end;

                        start = res - mh_range / 2;
                        end = res + mh_range / 2;
                        if (start < 0)
                            start = 0;
                        if (end > n_residues - 1)
                            end = n_residues - 1;
                        //printf("block: %d, start: %d, end: %d\n", b, start, end);


                        __syncthreads();

                        for (int tid = t; tid < n_energies; tid += stride)
                        {
                            // assign each thread (tid) an ij pair
                            int i = -1, tidc = tid, n_resc = n_residues - 1, restart_tid = -(n_resc + 1);
                            while (tidc >= 0)
                            {
                                restart_tid += n_resc + 1;
                                tidc -= n_resc;
                                n_resc--;
                                i++;
                            }
                            int j = tid - restart_tid + i + 1;

                            // we can calculate energy for this ij pair
                            float dist;
                            int bin_index, address;
                            dist = sqrtf(powf(state_shared[CB[i] * 3] - state_shared[CB[j] * 3], 2) + powf(state_shared[CB[i] * 3 + 1] - state_shared[CB[j] * 3 + 1], 2) + powf(state_shared[CB[i] * 3 + 2] - state_shared[CB[j] * 3 + 2], 2)) * 10;
                            bin_index = int((dist - 2.0) / (20.0 / 64.0));
                            if (bin_index < 0)
                                bin_index = 0;
                            else if (bin_index > 63)
                                bin_index = 63;
                            address = i * 64 * n_residues + j * 64 + bin_index;
                            if (bin_index == 63)
                            {
                                sums_glob_state[t] += -logf(distogram[address] / 100);
                                if (i >= start && j < end)
                                    sums_state[t] += -logf(distogram[address] / 100);
                            }
                            else
                            {     
                                sums_glob_state[t] += -logf(distogram[address]);
                                if (i >= start && j < end)
                                    sums_state[t] += -logf(distogram[address]);
                            }
                         
                            // repeat for proposal
                            dist = sqrtf(powf(proposal_state_shared[CB[i] * 3] - proposal_state_shared[CB[j] * 3], 2) + powf(proposal_state_shared[CB[i] * 3 + 1] - proposal_state_shared[CB[j] * 3 + 1], 2) + powf(proposal_state_shared[CB[i] * 3 + 2] - proposal_state_shared[CB[j] * 3 + 2], 2)) * 10;
                            bin_index = int((dist - 2.0) / (20.0 / 64.0));
                            if (bin_index < 0)
                                bin_index = 0;
                            else if (bin_index > 63)
                                bin_index = 63;
                            address = i * 64 * n_residues + j * 64 + bin_index;
                            if (bin_index == 63)
                            {
                                sums_glob_prop_state[t] += -logf(distogram[address] / 100);
                                if (i >= start && j < end)
                                    sums_prop_state[t] += -logf(distogram[address] / 100);
                            }
                            else
                            {   
                                sums_glob_prop_state[t] += -logf(distogram[address]);
                                if (i >= start && j < end)
                                    sums_prop_state[t] += -logf(distogram[address]);
                            }
                        }

                        __syncthreads();

                        // implement parallel reduction operation to sum all energies arrays, https://developer.download.nvidia.com/assets/cuda/files/reduction.pdf
                        for (int s = n_threads_const / 2; s > 0; s /= 2)
                        {
                            if (t < s)
                            {
                                sums_state[t] += sums_state[t + s];
                                sums_prop_state[t] += sums_prop_state[t + s];
                                sums_glob_state[t] += sums_glob_state[t + s];
                                sums_glob_prop_state[t] += sums_glob_prop_state[t + s];
                            }
                            __syncthreads();
                        }
                        
                     
                        // sequential summing [testing]
                        /*for (int s = 1; s < stride; s++)
                        {
                            sums_state[0] += sums_state[s];
                            sums_prop_state[0] += sums_prop_state[s];
                            sums_glob_state[0] += sums_glob_state[s];
                            sums_glob_prop_state[0] += sums_glob_prop_state[s];
                        }*/
                        // ------------------------------ end energy ------------------------------------------------------------------------------------------------------------------
                        __syncthreads();
                        // ------------------------------ criterion -----------------------------------------------------------------------------------------------------------------------
                        
                        if (t == 0)
                        {
                            float pi = expf(-1 * sums_state[0] * beta);
                            float pf = expf(-1 * sums_prop_state[0] * beta);

                            accept = false;
                            lowest = false;

                            float f = fabsf(pf / (pi+0.0000000000000001)) - rng_shared[0];
                            if (sums_prop_state[0] < sums_state[0] || f > 0)
                            {
                                // accept state
                                n_accepted++;
                                accept = true;
                                if (sums_glob_prop_state[0] < min_energy[b])
                                {
                                    lowest = true;
                                    min_energy[b] = sums_glob_prop_state[0];
                                    if (min_energy[b] <= *energy_target )
                                    {    
                                        *global_halt=true;
                                        break;
                                    }
                                 }
                                e_history[proposal + b * n_proposals] = sums_glob_prop_state[0];
                            }
                            else
                                e_history[proposal + b * n_proposals] = sums_glob_state[0];

                            proposal++;
                        }
                        __syncthreads();

                        if (accept)
                        {
                            for (int tid = t; tid < n_atoms * 3; tid += stride)
                                state_shared[tid] = proposal_state_shared[tid];
                            __syncthreads();
                            if (lowest)
                                for (int tid = t; tid < n_atoms * 3; tid += stride)
                                    min_state[tid + b * n_atoms * 3] = state_shared[tid];
                        }

                        // ------------------------------ end criterion ------------------------------------------------------------------------------------------------------------------

                        sums_state[t] = 0;
                        sums_prop_state[t] = 0;
                        sums_glob_prop_state[t] = 0;
                        sums_glob_state[t] = 0;
                    }
                }
            }
           
        }
        
        //--------------------------------------------------------------------------------------------------------
        // set each state to the overall min_state among all blocks (parallel-tempering style metropolis). Uncomment below code. 
        //--------------------------------------------------------------------------------------------------------
        /*
        if (cycle==0)
            beta = beta * (float)b/2.5;   
        if (t == 0 && b == 0)
        {
            *min_reset_energy = 100000;
            *min_reset_ind = 0;
            for (int i = 0; i < n_blocks_const; i++)
                if (min_energy[i] < *min_reset_energy)
                {
                    *min_reset_ind = i;
                    *min_reset_energy = min_energy[i];
                }
            for (int i = 0; i < n_blocks_const; i++)
                min_energy[i] = *min_reset_energy;
            printf("cycle: %d, min energy reset: %f\n", cycle, *min_reset_energy);
        }
        
        __syncthreads();

        for (int tid = t; tid < n_atoms * 3; tid += stride)
        {
            state_shared[tid] = min_state[tid + *min_reset_ind * n_atoms * 3];
            proposal_state_shared[tid] = min_state[tid + *min_reset_ind * n_atoms * 3];
            min_state[tid + b * n_atoms * 3] = state_shared[tid];
        }
        */
        //--------------------------------------------------------------------------------------------------------
        


    }
 
    __syncthreads();
     
    // write min_state and min_energy to global memory
    
    if (t == 0)
    {
        for (ii = 0; ii < n_atoms * 3; ii++)
            state[ii + b * n_atoms * 3] = min_state[ii + b * n_atoms * 3];
        energy[b] = min_energy[b];
        //printf("[log] Final min_energy: %f, n_accepted: %d, n_samples: %d, noise-var: %f, block: %d\n", min_energy[b], n_accepted, proposal, noise_var, b);
    }
 
    __syncthreads();
    float avg_min_energies = 0;
    if (t==0 & b == 0)
    {
        for (ii = 0; ii < n_blocks_const; ii++)
            avg_min_energies += min_energy[ii];
        avg_min_energies = avg_min_energies/((float)(n_blocks*1.0));
        printf("[log] average min_energy: %f\n", avg_min_energies);
    }
    
}

linenum: 0 number of tokens : 107584
linenum: 1 number of tokens : 492
linenum: 2 number of tokens : 41
linenum: 3 number of tokens : 41
linenum: 4 number of tokens : 41
linenum: 5 number of tokens : 41
[log] successfully read in the data!
[log] bytes of dynamic shared memory: 12128
[log] average min_energy: 2873.045654
Elapsed time in milliseconds: 1335 ms
[log] host min_energy: 2704.81
[log] bytes of dynamic shared memory: 12128
[log] average min_energy: 2882.428711
Elapsed time in milliseconds: 1344 ms
[log] host min_energy: 2665.35
[log] bytes of dynamic shared memory: 12128
[log] average min_energy: 2875.629150
Elapsed time in milliseconds: 1324 ms
[log] host min_energy: 2672.83
[log] bytes of dynamic shared memory: 12128
[log] average min_energy: 2883.456055
Elapsed time in milliseconds: 1323 ms
[log] host min_energy: 2678.92
[log] bytes of dynamic shared memory: 12128
[log] average min_energy: 2878.892578
Elapsed time in milliseconds: 1321 ms
[log] host min_energy: 2723.34
[log]

#RNG Efficiency Tester

In [ ]:
%%cu
// RNG tester

#include "cuda_runtime.h"
#include "device_launch_parameters.h"
//#include "data_header.h"
#include "cuda.h"
#ifndef __CUDACC__  
#define __CUDACC__
#endif
#include <curand.h>
#include <curand_kernel.h>

#include <stdio.h>
#include <string>
#include <vector>
#include <iostream>
#include <fstream>
#include <random>
#include <chrono>


using namespace std;

__global__ void rng_tester(int* rng, float*result);

static const int n_threads_const = 512;
static const int n_blocks_const = 50;
static const int n_rngs = 2;
static const int n_runs = 25;
static const int n_iter_per_run = 10000000;


int main()
{
    cudaError_t cudaStatus;
    float avg_time = 0;
    for (int run = 0; run < n_runs; run++)
    {
        // initialize RNG
        int rand_max = (1u << 31) - 1; // 2^32-1, should fill a 32 bit register
        std::default_random_engine generator;
        std::uniform_int_distribution<int> distribution(0, rand_max);
        generator.seed(std::chrono::system_clock::now().time_since_epoch().count()); 
        vector<int> build_rng;
        for (int i = 0; i < n_rngs * n_blocks_const; i++)
            build_rng.push_back(distribution(generator));
        int* rng = &build_rng[0];

        // reserve memory on device
        int *dev_rng;
        float* dev_result;
        float * zero = new float(0);

        cudaStatus = cudaMalloc((void**)&dev_rng, n_rngs * n_blocks_const * sizeof(int));
        cudaStatus = cudaMalloc((void**)&dev_result, sizeof(float));

        // fill memory on device
        cudaStatus = cudaMemcpy(dev_rng, rng, n_rngs * n_blocks_const * sizeof(int), cudaMemcpyHostToDevice);
        cudaStatus = cudaMemcpy(dev_result, zero, sizeof(float), cudaMemcpyHostToDevice);

        if (cudaStatus != cudaSuccess) {
            fprintf(stderr, "cudaMemcpy failed!");
        }
       
        auto start = chrono::steady_clock::now();
        rng_tester << < n_blocks_const, n_threads_const >> > (dev_rng, dev_result);

        // Check for any errors launching the kernel
        cudaStatus = cudaGetLastError();
        if (cudaStatus != cudaSuccess) {
            fprintf(stderr, "mcmc_block kernel launch failed: %s\n", cudaGetErrorString(cudaStatus));
        }

        // cudaDeviceSynchronize waits for the kernel to finish
        cudaStatus = cudaDeviceSynchronize();
        if (cudaStatus != cudaSuccess) {
            fprintf(stderr, "cudaDeviceSynchronize returned error code %d after launching kernel!\n", cudaStatus);
        }

        auto end = chrono::steady_clock::now();
        int time_ms = chrono::duration_cast<chrono::milliseconds>(end - start).count();
        cout << "Elapsed time in milliseconds: " << time_ms << " ms" << endl;
        avg_time += time_ms;
       
    }
    cout << "[log] average time over " << n_runs << " runs is : "<< avg_time / n_runs << "ms" << endl;



    // cudaDeviceReset must be called before exiting in order for profiling and
    // tracing tools such as Nsight and Visual Profiler to show complete traces.
    cudaStatus = cudaDeviceReset();
    if (cudaStatus != cudaSuccess) {
        fprintf(stderr, "cudaDeviceReset failed!");
        return 1;
    }

    return 0;
}

// kernel
__global__ void rng_tester(int* rng, float* result)
{
    int t = threadIdx.x;
    int stride = blockDim.x;
    int b = blockIdx.x;
    int n_blocks = gridDim.x;

    clock_t start_time = clock(); 
    curandState_t custate;
    curandStatePhilox4_32_10_t custate_philox;
    curandStateMRG32k3a_t custate_mrg32k3a;
    curandStateXORWOW_t custate_xorwow;
    
    __shared__ float rng_shared[n_rngs]; 
    __shared__ int rng_stored[n_rngs];
 
    if (t == 0)
        for (int ii = 0; ii < n_rngs; ii++)
            rng_stored[ii] = rng[ii + b * n_rngs];
    if (t < n_rngs)
    {
        curand_init(rng_stored[t],        /* the seed controls the sequence of random values that are produced */
              0,                                /* the sequence number is only important with multiple cores */
              0,                                /* the offset is how much extra we advance in the sequence for each call, can be 0 */
              &custate_xorwow);
    }
    // ------------------------------ pbit_logic: rng --------------------------------------------------------------------------------------------------------------
    for (int ii = 0; ii < n_iter_per_run; ii++)
    {
        if (t == 0)
            rng_shared[0] = curand_uniform(&custate_xorwow);
        if (t == 1)
        {    
            rng_shared[1] = curand_normal(&custate_xorwow);
            //printf("prop %d : rng: %f\n", ii, rng_shared[1]);
        }
        *result *= (rng_shared[0] + rng_shared[1]);
    }
    
}

Elapsed time in milliseconds: 3763 ms
Elapsed time in milliseconds: 3741 ms
Elapsed time in milliseconds: 3741 ms
Elapsed time in milliseconds: 3741 ms
Elapsed time in milliseconds: 3741 ms
Elapsed time in milliseconds: 3741 ms
Elapsed time in milliseconds: 3741 ms
Elapsed time in milliseconds: 3741 ms
Elapsed time in milliseconds: 3741 ms
Elapsed time in milliseconds: 3741 ms
Elapsed time in milliseconds: 3741 ms
Elapsed time in milliseconds: 3741 ms
Elapsed time in milliseconds: 3741 ms
Elapsed time in milliseconds: 3741 ms
Elapsed time in milliseconds: 3741 ms
Elapsed time in milliseconds: 3741 ms
Elapsed time in milliseconds: 3741 ms
Elapsed time in milliseconds: 3741 ms
Elapsed time in milliseconds: 3741 ms
Elapsed time in milliseconds: 3741 ms
Elapsed time in milliseconds: 3741 ms
Elapsed time in milliseconds: 3741 ms
Elapsed time in milliseconds: 3741 ms
Elapsed time in milliseconds: 3741 ms
Elapsed time in milliseconds: 3741 ms
[log] average time over 25 runs is : 3741.88ms

